# Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
) # 프롬프트를 여러 개 관리하기 위함

In [6]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonation a {character}
    """
)

example = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Huamn: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

C:\Users\yunyg\AppData\Local\Temp\ipykernel_16680\3577527870.py:42: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrr! That be a fine question, matey! I be lovin' me hearty, salted fish and a splash o' rum to wash it down! Yarrr!

AIMessage(content="Arrr! That be a fine question, matey! I be lovin' me hearty, salted fish and a splash o' rum to wash it down! Yarrr!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--da5c7385-e5c2-4fa3-8e79-15ffbb5ce942-0')

# Serialization and Composition 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
    streaming=True,
    callbacks= [
        StreamingStdOutCallbackHandler()
    ]
)

prompt = load_prompt("./prompt.json")

# prompt.format(country = "Germmany")

'What is the capital of Germmany'

In [8]:
prompt.invoke({
    "country" : "Germany"
})

StringPromptValue(text='What is the capital of Germany')

In [ ]:
chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'service_tier': 'default'}, id='run--f684fdae-9905-4153-b732-e210a2926246-0')

# Caching

In [1]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

C:\Users\yunyg\AppData\Local\Temp\ipykernel_10000\77896868.py:13: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("What is the capital of Japan")


'The capital of Japan is Tokyo.'

In [2]:
chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [3]:
chat.predict("What is the capital of Korea")

'The capital of South Korea is Seoul.'

# Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(True)

chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipt for soju")
    b = chat.predict("What is the recipt for bread")

    print(usage)
    print(usage.total_cost) # 전체비용
    print(usage.total_tokens) # 토큰 총합
    print(usage.prompt_tokens) #프롬프트에 사용된 토큰 수
    print(usage.completion_tokens) # 모델이 쓴 토큰

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipt for soju"
  ]
}
[llm/end] [llm:ChatOpenAI] [3.59s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Certainly! Here's a simple homemade recipe to make traditional Korean soju:\n\n**Homemade Soju Recipe**\n\n**Ingredients:**\n- 1 kg (2.2 lbs) of rice (preferably short-grain or sticky rice)\n- 1.5 liters of distilled water\n- 200 grams of nuruk (a traditional Korean fermentation starter)\n- Optional: sugar or sweeteners (if desired)\n\n**Equipment:**\n- Large fermentation container or jar\n- Steamer or large pot\n- Cheesecloth or cloth for covering\n- Sieve or strainer\n- Bottles for storage\n\n**Instructions:**\n\n1. **Prepare the Rice:**\n   - Rinse the rice thoroughly until the water runs clear.\n   - Soak the rice in water for about 4-6 hours or overnight.\n   - Drain the rice.\n\n2. **Cook the Rice:**\n   - Steam the rice until fully cooked and st